In [1]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten

model_new = Sequential([
  Conv2D(16, (5, 5), input_shape=(100, 100, 3), activation='relu' ),
  MaxPooling2D(pool_size=(2, 2), strides = 2),
  Conv2D(32, (5, 5), activation = 'relu'),
  MaxPooling2D(pool_size=(2, 2), strides = 2),
  Conv2D(64, (5, 5), activation = 'relu'),
  MaxPooling2D(pool_size=(2, 2), strides = 2),
  Conv2D(128, (5, 5), activation = 'relu'),
  MaxPooling2D(pool_size=(2, 2), strides = 2),
  Flatten(),
  Dense(1024, activation = 'relu'),
  Dense(256, activation = 'relu'),
  Dense(120, activation = 'softmax')
])

model_new.summary()

Using TensorFlow backend.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 16)        1216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 44, 44, 32)        12832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 64)        51264     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 5, 5, 128)        

In [2]:
model_new.load_weights('model_fruits.h5')
print('Loaded Weights into the new Model')

Loaded Weights into the new Model


In [3]:
model_new.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics=['accuracy'])

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)
test_dir = r"C:\Users\Sagar Mishra\jupyter notebooks\fruits-360_dataset\fruits-360\test-multiple_fruits"
test_generator = test_datagen.flow_from_directory(
test_dir,
target_size = (100, 100))

Found 0 images belonging to 0 classes.


In [6]:
import os
import imageio
import tensorflow as tf
import re
import numpy as np

test_names = os.listdir(test_dir)
names = test_names
for value in range(len(test_names)):
  test_names[value] = os.path.join(test_dir, test_names[value])

X = []
Y = []

for value in range(len(test_names)):
  numpy_image = imageio.imread(test_names[value])
  numpy_image = numpy_image.astype('float32')
  numpy_image /= 255.0
  tensor_image = tf.convert_to_tensor(numpy_image)
  X.append(tf.image.resize_with_pad(tensor_image, 100, 100))
  regexName  = re.compile(r'_([a-zA-Z]+){1,10}')
  Name = re.findall(regexName, names[value])
  regexNamei = re.compile(r'^([a-zA-Z]+)_')
  Namei = re.findall(regexNamei, names[value])
  Y.append(Namei + Name)
  
X = np.asarray(X)
Y = np.asarray(Y)

print(X.shape)
print(Y.shape)

(106, 100, 100, 3)
(106,)


In [12]:
pred = model_new.predict(x = X, batch_size = 10, verbose = 1 )
print(pred.shape) 


106/106 [==============================] - 0s 3ms/step
(106, 120)


In [16]:
print(type(pred))

<class 'numpy.ndarray'>


In [14]:
max_value = 0.0
index = 0
for i, values in enumerate(pred[0]):
  if max_value < values:
    max_value = values
    index = i

print(index, max_value)

12 1.0


In [15]:
train_path = 'C:\\Users\\Sagar Mishra\\jupyter notebooks\\fruits-360_dataset\\fruits-360\\Training'

all_avail_names = os.listdir(train_path)
dict_of_names = {}

for i,name in enumerate(all_avail_names):
  dict_of_names[i] = name
  
print(dict_of_names)

{0: 'Apple Braeburn', 1: 'Apple Crimson Snow', 2: 'Apple Golden 1', 3: 'Apple Golden 2', 4: 'Apple Golden 3', 5: 'Apple Granny Smith', 6: 'Apple Pink Lady', 7: 'Apple Red 1', 8: 'Apple Red 2', 9: 'Apple Red 3', 10: 'Apple Red Delicious', 11: 'Apple Red Yellow 1', 12: 'Apple Red Yellow 2', 13: 'Apricot', 14: 'Avocado', 15: 'Avocado ripe', 16: 'Banana', 17: 'Banana Lady Finger', 18: 'Banana Red', 19: 'Beetroot', 20: 'Blueberry', 21: 'Cactus fruit', 22: 'Cantaloupe 1', 23: 'Cantaloupe 2', 24: 'Carambula', 25: 'Cauliflower', 26: 'Cherry 1', 27: 'Cherry 2', 28: 'Cherry Rainier', 29: 'Cherry Wax Black', 30: 'Cherry Wax Red', 31: 'Cherry Wax Yellow', 32: 'Chestnut', 33: 'Clementine', 34: 'Cocos', 35: 'Dates', 36: 'Eggplant', 37: 'Ginger Root', 38: 'Granadilla', 39: 'Grape Blue', 40: 'Grape Pink', 41: 'Grape White', 42: 'Grape White 2', 43: 'Grape White 3', 44: 'Grape White 4', 45: 'Grapefruit Pink', 46: 'Grapefruit White', 47: 'Guava', 48: 'Hazelnut', 49: 'Huckleberry', 50: 'Kaki', 51: 'Kiwi'

In [18]:
predictions = []
for prediction in pred:
  predictions.append(np.argmax(prediction))
  
predicted_names = []
for values in predictions:
  predicted_names.append(dict_of_names[values])

for i, ans in enumerate(predicted_names):
  print(i, ans)

0 Apple Red Yellow 2
1 Cauliflower
2 Pepper Green
3 Strawberry
4 Cauliflower
5 Plum
6 Plum
7 Pepper Green
8 Strawberry
9 Strawberry
10 Apple Crimson Snow
11 Strawberry
12 Strawberry
13 Strawberry
14 Mangostan
15 Carambula
16 Carambula
17 Carambula
18 Carambula
19 Lemon Meyer
20 Strawberry
21 Mangostan
22 Plum
23 Eggplant
24 Pepper Green
25 Strawberry
26 Avocado
27 Strawberry
28 Pepper Green
29 Carambula
30 Strawberry
31 Strawberry
32 Strawberry
33 Cauliflower
34 Plum
35 Plum
36 Strawberry
37 Pepper Green
38 Pepper Green
39 Pineapple
40 Mangostan
41 Eggplant
42 Cauliflower
43 Carambula
44 Strawberry
45 Strawberry
46 Strawberry
47 Strawberry
48 Pineapple Mini
49 Strawberry
50 Cherry 1
51 Apple Red Yellow 2
52 Cauliflower
53 Carambula
54 Carambula
55 Dates
56 Pepper Yellow
57 Cauliflower
58 Apple Red Yellow 2
59 Pineapple Mini
60 Quince
61 Strawberry
62 Pineapple
63 Strawberry
64 Carambula
65 Pepper Green
66 Cauliflower
67 Pepper Green
68 Pepper Green
69 Strawberry
70 Strawberry
71 Strawb

In [21]:
import warnings
warnings.simplefilter('ignore')

test_path = 'C:\\Users\\Sagar Mishra\\jupyter notebooks\\fruits-360_dataset\\fruits-360\\Test'
test_datagen = ImageDataGenerator(1./255)
test_generator = test_datagen.flow_from_directory(
directory = test_path,
target_size = (100, 100))

model_new.evaluate_generator(test_generator)

Found 20622 images belonging to 120 classes.


[363.60247802734375, 0.9675104022026062]